<a href="https://colab.research.google.com/github/vanikk06/NOMURA_PROJECT/blob/master/this!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving nomura_global_equity.csv to nomura_global_equity.csv


In [0]:
def NomuraData():
    HData = open("nomura_global_equity.csv").readlines()
    MData = [line.strip(",\n").split(";") for line in HData]
    for i in range(len(MData)):
        MData[i][1] = float(MData[i][1])
        MData[i][2] = float(MData[i][2])
    return MData

In [0]:
Data = NomuraData()

In [0]:
dt = []
for i in range(len(Data)):
    dt.append(Data[i][0])

for i in dt:
    df = [line.split("/") for line in dt]

for i in range(len(df)):
    df[i][0] = int(df[i][0])
    df[i][1] = int(df[i][1])
    df[i][2] = int(df[i][2])

for i in range(len(df)):
  df[i].append(Data[i][0])
  df[i].append(Data[i][1])
  df[i].append(Data[i][2])

In [6]:
data = pd.DataFrame(df,columns = ["Day","Month","Year","Date","Price","10D Return"])
data

,Day,Month,Year,Date,Price,10D Return
0,4,1,1997,4/1/1997,12.14,0.017603
1,6,1,1997,6/1/1997,12.13,0.011676
2,7,1,1997,7/1/1997,12.12,0.013378
3,8,1,1997,8/1/1997,12.17,0.018410
4,9,1,1997,9/1/1997,12.20,0.020921
...,...,...,...,...,...,...
5872,8,4,2020,8/4/2020,17.83,0.181577
5873,9,4,2020,9/4/2020,17.99,0.106396
5874,13,4,2020,13/4/2020,17.92,0.081473
5875,14,4,2020,14/4/2020,18.42,0.054983


In [7]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data

,Day,Month,Year,Date,Price,10D Return
0,4,1,1997,1997-01-04,12.14,0.017603
1,6,1,1997,1997-01-06,12.13,0.011676
2,7,1,1997,1997-01-07,12.12,0.013378
3,8,1,1997,1997-01-08,12.17,0.018410
4,9,1,1997,1997-01-09,12.20,0.020921
...,...,...,...,...,...,...
5872,8,4,2020,2020-04-08,17.83,0.181577
5873,9,4,2020,2020-04-09,17.99,0.106396
5874,13,4,2020,2020-04-13,17.92,0.081473
5875,14,4,2020,2020-04-14,18.42,0.054983


In [0]:
data.insert(6, "600roll_mean", 0)
data.insert(7, "600roll_std", 0)

In [9]:
data['600roll_mean'] = data['10D Return'].rolling(window = 600, min_periods = 1).mean()
data['600roll_std'] = data['10D Return'].rolling(window = 600, min_periods = 1).std()
data

,Day,Month,Year,Date,Price,10D Return,600roll_mean,600roll_std
0,4,1,1997,1997-01-04,12.14,0.017603,0.017603,NaN
1,6,1,1997,1997-01-06,12.13,0.011676,0.014639,0.004191
2,7,1,1997,1997-01-07,12.12,0.013378,0.014219,0.003052
3,8,1,1997,1997-01-08,12.17,0.018410,0.015267,0.003256
4,9,1,1997,1997-01-09,12.20,0.020921,0.016398,0.003787
...,...,...,...,...,...,...,...,...
5872,8,4,2020,2020-04-08,17.83,0.181577,0.003239,0.036750
5873,9,4,2020,2020-04-09,17.99,0.106396,0.003401,0.036990
5874,13,4,2020,2020-04-13,17.92,0.081473,0.003509,0.037123
5875,14,4,2020,2020-04-14,18.42,0.054983,0.003552,0.037168


In [0]:
data.insert(8, "State", np.nan) 

In [11]:
for i in range(len(data)):
  if data['10D Return'][i] > (data['600roll_mean'][i] + 3*data['600roll_std'][i]):
    data['State'][i] = 'extremely high'
  elif (data['600roll_mean'][i] + 3*data['600roll_std'][i]) >= data['10D Return'][i] > (data['600roll_mean'][i] + 2*data['600roll_std'][i]):
    data['State'][i] = 'very high'
  elif (data['600roll_mean'][i] + 2*data['600roll_std'][i]) >= data['10D Return'][i] > (data['600roll_mean'][i] + data['600roll_std'][i]):
    data['State'][i] = 'high'
  elif (data['600roll_mean'][i] + data['600roll_std'][i]) >= data['10D Return'][i] >= (data['600roll_mean'][i] - data['600roll_std'][i]):
    data['State'][i] = 'mid'
  elif (data['600roll_mean'][i] - data['600roll_std'][i]) > data['10D Return'][i] >= (data['600roll_mean'][i] - 2*data['600roll_std'][i]):
    data['State'][i] = 'low'
  elif (data['600roll_mean'][i] - 2*data['600roll_std'][i]) > data['10D Return'][i] >= (data['600roll_mean'][i] - 3*data['600roll_std'][i]):
    data['State'][i] = 'very low'
  elif (data['600roll_mean'][i] - 3*data['600roll_std'][i]) > data['10D Return'][i]:
    data['State'][i] = 'extremely low'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is try

In [12]:
data

,Day,Month,Year,Date,Price,10D Return,600roll_mean,600roll_std,State
0,4,1,1997,1997-01-04,12.14,0.017603,0.017603,NaN,NaN
1,6,1,1997,1997-01-06,12.13,0.011676,0.014639,0.004191,mid
2,7,1,1997,1997-01-07,12.12,0.013378,0.014219,0.003052,mid
3,8,1,1997,1997-01-08,12.17,0.018410,0.015267,0.003256,mid
4,9,1,1997,1997-01-09,12.20,0.020921,0.016398,0.003787,high
...,...,...,...,...,...,...,...,...,...
5872,8,4,2020,2020-04-08,17.83,0.181577,0.003239,0.036750,extremely high
5873,9,4,2020,2020-04-09,17.99,0.106396,0.003401,0.036990,very high
5874,13,4,2020,2020-04-13,17.92,0.081473,0.003509,0.037123,very high
5875,14,4,2020,2020-04-14,18.42,0.054983,0.003552,0.037168,high


In [0]:
Invest = []

for i in range(len(data)):
  if data['State'][i] == 'very low':
    Invest.append(1)
  elif data['State'][i] == 'extremely low':
    Invest.append(1)
  else:
    Invest.append(0)

In [14]:
data['Invest'] = Invest
data

,Day,Month,Year,Date,Price,10D Return,600roll_mean,600roll_std,State,Invest
0,4,1,1997,1997-01-04,12.14,0.017603,0.017603,NaN,NaN,0
1,6,1,1997,1997-01-06,12.13,0.011676,0.014639,0.004191,mid,0
2,7,1,1997,1997-01-07,12.12,0.013378,0.014219,0.003052,mid,0
3,8,1,1997,1997-01-08,12.17,0.018410,0.015267,0.003256,mid,0
4,9,1,1997,1997-01-09,12.20,0.020921,0.016398,0.003787,high,0
...,...,...,...,...,...,...,...,...,...,...
5872,8,4,2020,2020-04-08,17.83,0.181577,0.003239,0.036750,extremely high,0
5873,9,4,2020,2020-04-09,17.99,0.106396,0.003401,0.036990,very high,0
5874,13,4,2020,2020-04-13,17.92,0.081473,0.003509,0.037123,very high,0
5875,14,4,2020,2020-04-14,18.42,0.054983,0.003552,0.037168,high,0


In [15]:
sum(data['Invest'])

251

In [16]:
for i in range(len(data)):
  if data['Invest'][i] == 1 and data['Invest'][i-5] == 1:
    if data['Price'][i] <= data['Price'][i-5]:
      data['Invest'][i-5] = 0
    elif data['Price'][i] > data['Price'][i-5]:
      data['Invest'][i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
for i in range(len(data)):
  if data['Invest'][i] == 1 and data['Invest'][i-4] == 1:
    if data['Price'][i] <= data['Price'][i-4]:
      data['Invest'][i-4] = 0
    elif data['Price'][i] > data['Price'][i-4]:
      data['Invest'][i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
for i in range(len(data)):
  if data['Invest'][i] == 1 and data['Invest'][i-3] == 1:
    if data['Price'][i] <= data['Price'][i-3]:
      data['Invest'][i-3] = 0
    elif data['Price'][i] > data['Price'][i-3]:
      data['Invest'][i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
for i in range(len(data)):
  if data['Invest'][i] == 1 and data['Invest'][i-2] == 1:
    if data['Price'][i] <= data['Price'][i-2]:
      data['Invest'][i-2] = 0
    elif data['Price'][i] > data['Price'][i-2]:
      data['Invest'][i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
for i in range(len(data)):
  if data['Invest'][i] == 1 and data['Invest'][i-1] == 1:
    if data['Price'][i] <= data['Price'][i-1]:
      data['Invest'][i-1] = 0
    elif data['Price'][i] > data['Price'][i-1]:
      data['Invest'][i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
data.insert(10, "In_time", 0)

In [22]:
for i in range(len(data)):
  if data['Invest'][i] == 1:
    if data['Price'][i] <= data['Price'][i+1]:
      data['In_time'][i+1] = 1
    elif data['Price'][i] > data['Price'][i+1]:
      if data['Price'][i] <= data['Price'][i+2]:
        data['In_time'][i+2] = 1
      elif data['Price'][i] > data['Price'][i+2]:
        if data['Price'][i] <= data['Price'][i+3]:
          data['In_time'][i+3] = 1
        elif data['Price'][i] > data['Price'][i+3]:
          if data['Price'][i] <= data['Price'][i+4]:
            data['In_time'][i+4] = 1
          elif data['Price'][i] > data['Price'][i+4]:
            if data['Price'][i] <= data['Price'][i+5]:
              data['In_time'][i+5] = 1
            elif data['Price'][i] > data['Price'][i+5]:
              if data['Price'][i] <= data['Price'][i+6]:
                data['In_time'][i+6] = 1
              elif data['Price'][i] > data['Price'][i+6]:
                if data['Price'][i] <= data['Price'][i+7]:
                  data['In_time'][i+7] = 1
                elif data['Price'][i] > data['Price'][i+7]:
                  if data['Price'][i] <= data['Price'][i+8]:
                    data['In_time'][i+8] = 1
                  elif data['Price'][i] > data['Price'][i+8]:
                    if data['Price'][i] <= data['Price'][i+9]:
                      data['In_time'][i+9] = 1
                    elif data['Price'][i] > data['Price'][i+9]:
                      if data['Price'][i] <= data['Price'][i+10]:
                        data['In_time'][i+10] = 1
                      elif data['Price'][i] > data['Price'][i+10]:
                        data['In_time'][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
In_Price = []

for i in range(len(data)):
  if data['In_time'][i] == 1:
    In_Price.append(data['Price'][i])
  elif data['In_time'][i] == 0:
    In_Price.append(np.nan)

In [24]:
data['In_Price'] = In_Price
data

,Day,Month,Year,Date,Price,10D Return,600roll_mean,600roll_std,State,Invest,In_time,In_Price
0,4,1,1997,1997-01-04,12.14,0.017603,0.017603,NaN,NaN,0,0,NaN
1,6,1,1997,1997-01-06,12.13,0.011676,0.014639,0.004191,mid,0,0,NaN
2,7,1,1997,1997-01-07,12.12,0.013378,0.014219,0.003052,mid,0,0,NaN
3,8,1,1997,1997-01-08,12.17,0.018410,0.015267,0.003256,mid,0,0,NaN
4,9,1,1997,1997-01-09,12.20,0.020921,0.016398,0.003787,high,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5872,8,4,2020,2020-04-08,17.83,0.181577,0.003239,0.036750,extremely high,0,0,NaN
5873,9,4,2020,2020-04-09,17.99,0.106396,0.003401,0.036990,very high,0,0,NaN
5874,13,4,2020,2020-04-13,17.92,0.081473,0.003509,0.037123,very high,0,0,NaN
5875,14,4,2020,2020-04-14,18.42,0.054983,0.003552,0.037168,high,0,0,NaN


In [25]:
sum(data['In_time'])

52

In [26]:
fig = go.Figure()
fig.update_layout(title = '價格走勢', 
                  xaxis_title = 'Date',
                  yaxis_title = 'Price')

fig.add_trace(go.Scatter(x=data['Date'], y=data['Price'],
                    mode='lines',
                    name='lines'))

fig.add_trace(go.Scatter(mode='markers',x=data['Date'],y=data['In_Price'],opacity=0.6,
                         marker=dict(color='red',line=dict(color='MediumPurple',width=1))
                         ,name='enter point'))

fig.show()

In [0]:
nian = []
yue = []
ri = []
invest_d = []

for i in range(1997, 2019+1):
  nian = data.loc[data['Year'] == i]
  for j in range(1, 12+1):
    yue = nian.loc[nian['Month'] == j] 
    ri = yue.values.tolist()
    invest_d.append(ri[0])
    ri = []
    yue = []

for i in range(2020, 2020+1):
  nian = data.loc[data['Year'] == i]
  for j in range(1, 4+1):
    yue = nian.loc[nian['Month'] == j] 
    ri = yue.values.tolist()
    invest_d.append(ri[0])
    ri = []
    yue = []

In [0]:
invest_d = invest_d[1:]

for i in range(len(invest_d)):
  invest_d[i][0] = int(invest_d[i][0])
  invest_d[i][1] = int(invest_d[i][1])
  invest_d[i][2] = int(invest_d[i][2])
  invest_d[i][6] = int(invest_d[i][6])

In [0]:
#假設每次投3000
per = 3000
totalinfix = 0
totaloutfix = 0
unit = 0

for i in range(len(invest_d)):
  unit += per/invest_d[i][4]
  totalinfix += per

totaloutfix = unit*data['Price'][5876]

In [30]:
earnforfixed = totaloutfix/totalinfix
earnforfixed

1.8013649029575491

In [0]:
#假設每次投3000 + 觸價3000
per = 3000
hit = 3000
totalin = 0
totalout = 0
unit = 0

for i in range(len(invest_d)):
  unit += per/invest_d[i][4]
  totalin += per

for i in range(len(df)):
  if data['Invest'][i] == 1:
    unit += hit/data['Price'][i]
    totalin += hit
  
totalout = unit*data['Price'][5876]

In [32]:
earnforv = totalout/totalin
earnforv

1.8163343851024338